# AI Safety Papers Visualization - Phase 1

This notebook implements Phase 1 of the AI Safety Papers visualization project:
1. Metadata Collection using arXiv OAI-PMH API
2. Abstract Embedding Generation using ModernBERT-large
3. Initial Clustering using UMAP and HDBSCAN

In [ ]:
# Install required packages
%pip install -r requirements.txt

## Configuration Parameters

In [ ]:
# @title Data Collection Parameters
# Set to 0 to fetch all CS papers (warning: this will take a long time)
# Set to a positive number to fetch only that many papers
num_papers = 100 # @param {type:"slider", min:0, max:10000, step:100}

## 1. Database Setup

In [ ]:
import sqlite3
import os

def create_database():
    """Create SQLite database with necessary tables"""
    conn = sqlite3.connect('papers.db')
    c = conn.cursor()
    
    # # Drop existing tables if they exist to ensure clean schema
    # c.execute('DROP TABLE IF EXISTS paper_authors')
    # c.execute('DROP TABLE IF EXISTS author_affiliations')
    # c.execute('DROP TABLE IF EXISTS authors')
    # c.execute('DROP TABLE IF EXISTS papers')
    # c.execute('DROP TABLE IF EXISTS pipeline_progress')
    # c.execute('DROP TABLE IF EXISTS cluster_descriptions')
    
    # Create papers table with all arXiv metadata fields
    c.execute('''
        CREATE TABLE IF NOT EXISTS papers (
            id TEXT PRIMARY KEY,           -- Required, maxOccurs=1
            created TEXT,                  -- Optional, maxOccurs=1
            updated TEXT,                  -- Optional, maxOccurs=1
            title TEXT,                    -- Optional, maxOccurs=1
            abstract TEXT,                 -- Optional, maxOccurs=1
            categories TEXT,               -- Optional, maxOccurs=1
            msc_class TEXT,                -- Optional, maxOccurs=1
            acm_class TEXT,                -- Optional, maxOccurs=1
            doi TEXT,                      -- Optional, maxOccurs=1
            license TEXT,                  -- Optional, maxOccurs=1
            -- Fields we add for our analysis
            full_text TEXT,                -- Only for relevant papers
            abstract_embedding BLOB,        -- For all CS papers
            full_text_embedding BLOB,       -- Only for relevant papers
            initial_cluster_id INTEGER,     -- From abstract clustering
            final_cluster_id INTEGER,       -- From final clustering
            is_relevant BOOLEAN,            -- Identified as AI Safety paper
            relevance_score FLOAT,          -- Confidence in relevance
            status TEXT,                    -- Track paper processing status
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create authors table matching arXiv schema
    c.execute('''
        CREATE TABLE IF NOT EXISTS authors (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            keyname TEXT NOT NULL,          -- Required, maxOccurs=1
            forenames TEXT,                 -- Optional, maxOccurs=1
            suffix TEXT,                    -- Optional, maxOccurs=1
            UNIQUE(keyname, forenames, suffix)  -- Avoid duplicate authors
        )
    ''')
    
    # Create author affiliations table (since it's maxOccurs=unbounded in schema)
    c.execute('''
        CREATE TABLE IF NOT EXISTS author_affiliations (
            author_id INTEGER,
            affiliation TEXT NOT NULL,
            PRIMARY KEY (author_id, affiliation),
            FOREIGN KEY (author_id) REFERENCES authors(id)
        )
    ''')
    
    # Create paper_authors junction table
    c.execute('''
        CREATE TABLE IF NOT EXISTS paper_authors (
            paper_id TEXT,
            author_id INTEGER,
            author_position INTEGER,        -- Track author order in paper
            PRIMARY KEY (paper_id, author_id),
            FOREIGN KEY (paper_id) REFERENCES papers(id),
            FOREIGN KEY (author_id) REFERENCES authors(id)
        )
    ''')
    
    # Create progress tracking table
    c.execute('''
        CREATE TABLE IF NOT EXISTS pipeline_progress (
            phase TEXT PRIMARY KEY,
            current_count INTEGER,
            total_count INTEGER,
            status TEXT,
            last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create cluster descriptions table
    c.execute('''
        CREATE TABLE IF NOT EXISTS cluster_descriptions (
            cluster_id INTEGER PRIMARY KEY,
            is_initial BOOLEAN,            -- True for abstract clusters, False for final
            description TEXT,              -- Generated by Gemini
            representative_papers TEXT,     -- JSON array of paper IDs
            paper_count INTEGER,
            created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
    ''')
    
    # Create indices for common queries after all tables are created
    c.execute("CREATE INDEX IF NOT EXISTS idx_status ON papers(status)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_initial_cluster_id ON papers(initial_cluster_id)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_final_cluster_id ON papers(final_cluster_id)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_is_relevant ON papers(is_relevant)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_categories ON papers(categories)")
    c.execute("CREATE INDEX IF NOT EXISTS idx_authors_names ON authors(keyname, forenames)")
    
    conn.commit()
    return conn

# Initialize database
conn = create_database()

## Load Existing Database

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Check if database exists in Drive
db_path = "/content/drive/MyDrive/ai-safety-papers/papers.db"
if os.path.exists(db_path):
    print(f"Found existing database at {db_path}")
    !cp "{db_path}" papers.db
    
    # Print existing data summary
    conn = sqlite3.connect('papers.db')
    c = conn.cursor()
    c.execute('SELECT COUNT(*) FROM papers')
    print(f"Database contains {c.fetchone()[0]} papers")
else:
    print("No existing database found in Drive. Will create new one.")

## 2. Metadata Harvesting

In [ ]:
from sickle import Sickle
from sickle.models import Record
from tqdm import tqdm

class ArxivRecord(Record):
    """Custom record class for arXiv metadata format"""
    def get_metadata(self):
        # Get the arXiv metadata namespace
        ns = {'arxiv': 'http://arxiv.org/OAI/arXiv/'}
        
        # Get the arXiv element which contains all metadata
        arxiv = self.xml.find('.//{http://arxiv.org/OAI/arXiv/}arXiv')
        if arxiv is None:
            raise ValueError("Could not find arXiv metadata element")
            
        metadata = {}
        
        # Required field - use identifier from header
        metadata['id'] = self.header.identifier
        if metadata['id'].startswith('oai:arXiv.org:'):
            metadata['id'] = metadata['id'].replace('oai:arXiv.org:', '')
        
        # Map arXiv metadata fields according to schema
        field_mapping = {
            'created': 'created',
            'updated': 'updated',
            'title': 'title',
            'abstract': 'abstract',
            'categories': 'categories',
            'msc_class': 'msc-class',
            'acm_class': 'acm-class',
            'report_no': 'report-no',
            'journal_ref': 'journal-ref',
            'doi': 'doi',
            'comments': 'comments',
            'license': 'license'
        }
        
        for field, xml_field in field_mapping.items():
            elem = arxiv.find(f'arxiv:{xml_field}', namespaces=ns)
            metadata[field] = elem.text if elem is not None and elem.text else None
        
        # Extract authors according to schema
        authors = []
        authors_elem = arxiv.find('arxiv:authors', namespaces=ns)
        if authors_elem is not None:
            for author_elem in authors_elem.findall('arxiv:author', namespaces=ns):
                if author_elem is None:
                    continue
                    
                author = {}
                
                # Required keyname field
                keyname_elem = author_elem.find('arxiv:keyname', namespaces=ns)
                if keyname_elem is None or not keyname_elem.text:
                    continue  # Skip authors without keyname
                author['keyname'] = keyname_elem.text
                
                # Optional author fields
                forenames_elem = author_elem.find('arxiv:forenames', namespaces=ns)
                author['forenames'] = forenames_elem.text if forenames_elem is not None and forenames_elem.text else None
                
                suffix_elem = author_elem.find('arxiv:suffix', namespaces=ns)
                author['suffix'] = suffix_elem.text if suffix_elem is not None and suffix_elem.text else None
                
                # Multiple affiliations possible
                author['affiliations'] = []
                for affiliation_elem in author_elem.findall('arxiv:affiliation', namespaces=ns):
                    if affiliation_elem is not None and affiliation_elem.text:
                        author['affiliations'].append(affiliation_elem.text)
                
                authors.append(author)
        
        metadata['authors'] = authors
        return metadata

def save_papers(papers, conn):
    """Save papers and authors to SQLite database"""
    c = conn.cursor()
    
    for paper in papers:
        try:
            # Insert paper with all fields
            c.execute('''
                INSERT OR IGNORE INTO papers (
                    id, created, updated, title, abstract, categories,
                    msc_class, acm_class, doi, license, status
                ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            ''', (
                paper['id'], 
                paper.get('created'),
                paper.get('updated'),
                paper.get('title'),
                paper.get('abstract'),
                paper.get('categories'),
                paper.get('msc_class'),
                paper.get('acm_class'),
                paper.get('doi'),
                paper.get('license'),
                'fetched'
            ))
            
            # Process authors
            for pos, author in enumerate(paper['authors'], 1):
                # Insert author if not exists
                c.execute('''
                    INSERT OR IGNORE INTO authors (keyname, forenames, suffix)
                    VALUES (?, ?, ?)
                ''', (
                    author['keyname'],
                    author.get('forenames'),
                    author.get('suffix')
                ))
                
                # Get author ID
                c.execute('''
                    SELECT id FROM authors 
                    WHERE keyname = ? AND 
                          COALESCE(forenames, '') = COALESCE(?, '') AND
                          COALESCE(suffix, '') = COALESCE(?, '')
                ''', (
                    author['keyname'],
                    author.get('forenames'),
                    author.get('suffix')
                ))
                author_id = c.fetchone()[0]
                
                # Create paper-author relationship
                c.execute('''
                    INSERT OR IGNORE INTO paper_authors (paper_id, author_id, author_position)
                    VALUES (?, ?, ?)
                ''', (paper['id'], author_id, pos))
                
                # Add any affiliations
                for affiliation in author.get('affiliations', []):
                    c.execute('''
                        INSERT OR IGNORE INTO author_affiliations (author_id, affiliation)
                        VALUES (?, ?)
                    ''', (author_id, affiliation))
                
        except sqlite3.IntegrityError as e:
            print(f"Error saving paper {paper['id']}: {str(e)}")
            continue
            
    conn.commit()

def fetch_cs_papers(start_index=0, max_results=None):
    """Fetch CS papers from arXiv using OAI-PMH with arXiv metadata format"""
    # Create Sickle client with retry configuration
    sickle = Sickle('http://export.arxiv.org/oai2',
                    max_retries=5,
                    retry_status_codes=[503],
                    default_retry_after=20)
    
    # Register our custom record class for arXiv metadata format
    sickle.class_mapping['ListRecords'] = ArxivRecord
    
    papers = []
    try:
        # Use ListRecords with arXiv metadata format
        records = sickle.ListRecords(
            metadataPrefix='arXiv',  # Use arXiv's native format
            set='cs',
            ignore_deleted=True
        )
        
        # Process records with progress bar
        with tqdm(desc="Fetching papers", unit=" papers") as pbar:
            for record in records:
                try:
                    metadata = record.get_metadata()
                    if metadata and metadata.get('id'):  # Only add if we have valid metadata
                        papers.append(metadata)
                        pbar.update(1)
                        
                        # Respect max_results limit if set
                        if max_results and len(papers) >= max_results:
                            break
                except Exception as e:
                    print(f"Error processing record: {str(e)}")
                    continue
                    
    except Exception as e:
        print(f"Error during harvesting: {str(e)}")
        
    return papers

In [ ]:
# Fetch papers using the slider value (None for all papers)
initial_papers = fetch_cs_papers(max_results=num_papers if num_papers > 0 else None)
save_papers(initial_papers, conn)

# Print summary
c = conn.cursor()
c.execute('SELECT COUNT(*) FROM papers')
print(f"Total papers in database: {c.fetchone()[0]}")

## Data Quality Check

In [ ]:
def inspect_papers(conn, limit=5):
    """Print detailed information for a sample of papers"""
    c = conn.cursor()
    
    # Get sample papers with their authors
    c.execute('''
        SELECT p.id, p.title, p.abstract, p.categories, p.created, p.updated,
               GROUP_CONCAT(a.keyname || COALESCE(', ' || a.forenames, '') || COALESCE(' ' || a.suffix, ''))
        FROM papers p
        LEFT JOIN paper_authors pa ON p.id = pa.paper_id
        LEFT JOIN authors a ON pa.author_id = a.id
        GROUP BY p.id
        LIMIT ?
    ''', (limit,))
    
    papers = c.fetchall()
    
    print(f"Inspecting {len(papers)} sample papers:\n")
    for paper in papers:
        print("=" * 80)
        print(f"ID: {paper[0]}")
        print(f"Title: {paper[1]}")
        print(f"Abstract: {paper[2][:200]}..." if paper[2] else "Abstract: None")
        print(f"Categories: {paper[3]}")
        print(f"Created: {paper[4]}")
        print(f"Updated: {paper[5]}")
        print(f"Authors: {paper[6]}")
        print()
    
    # Print detailed statistics
    print("\nDatabase Statistics:")
    
    # Paper statistics
    c.execute('SELECT COUNT(*) FROM papers')
    total_papers = c.fetchone()[0]
    print(f"\nPapers:")
    print(f"Total Papers: {total_papers}")
    
    # Author statistics
    c.execute('SELECT COUNT(*) FROM authors')
    total_authors = c.fetchone()[0]
    
    c.execute('SELECT COUNT(*) FROM paper_authors')
    total_author_links = c.fetchone()[0]
    
    c.execute('''
        SELECT COUNT(*) FROM 
        (SELECT author_id FROM paper_authors GROUP BY author_id HAVING COUNT(*) > 1)
    ''')
    authors_multiple_papers = c.fetchone()[0]
    
    c.execute('SELECT AVG(author_count) FROM (SELECT paper_id, COUNT(*) as author_count FROM paper_authors GROUP BY paper_id)')
    avg_authors_per_paper = c.fetchone()[0]
    
    print(f"\nAuthors:")
    print(f"Total Authors: {total_authors}")
    print(f"Total Author-Paper Links: {total_author_links}")
    print(f"Authors with Multiple Papers: {authors_multiple_papers}")
    print(f"Average Authors per Paper: {avg_authors_per_paper:.2f}")
    
    # Metadata field statistics
    print("\nMetadata Field Coverage:")
    fields = [
        'title', 'abstract', 'categories', 'created', 'updated',
        'msc_class', 'acm_class', 'doi', 'license'
    ]
    
    for field in fields:
        # Count non-null values
        c.execute(f'SELECT COUNT(*) FROM papers WHERE {field} IS NOT NULL')
        present = c.fetchone()[0]
        percentage = (present / total_papers) * 100 if total_papers > 0 else 0
        print(f"{field}: {present}/{total_papers} ({percentage:.1f}%)")
    
    # Category statistics
    print("\nTop Categories:")
    c.execute('''
        SELECT categories, COUNT(*) as count 
        FROM papers 
        WHERE categories IS NOT NULL 
        GROUP BY categories 
        ORDER BY count DESC 
        LIMIT 5
    ''')
    for category, count in c.fetchall():
        percentage = (count / total_papers) * 100
        print(f"{category}: {count} papers ({percentage:.1f}%)")

# Inspect sample papers
inspect_papers(conn) 

## Save Database to Drive

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create project directory if it doesn't exist
!mkdir -p "/content/drive/MyDrive/ai-safety-papers"

# Copy database to Drive
!cp papers.db "/content/drive/MyDrive/ai-safety-papers/papers.db"
print("Database saved to Google Drive at: /ai-safety-papers/papers.db") 